# Import dependencies and custom modules

In [12]:
# # # Clear the Python kernel's memory cache
# import gc
# gc.collect()

# Import all required dependencies
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime
import pvlib
from shapely.geometry import Polygon
import fiona
import folium
from folium import Choropleth
import branca.colormap as cm
import json
import requests
from folium.plugins import Fullscreen, MeasureControl
from pvlib.location import Location
import warnings
import rasterio
from rasterio.mask import mask
from shapely.ops import transform
import pyproj
from functools import partial
from pathlib import Path
warnings.filterwarnings('ignore')


# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


# Load Data

In [13]:
# Second cell - Set paths and load data
# Set base input directory
INPUT_DIR = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/06 Scripts/ResilienceHub/Input'

# Derive paths for input files
gdb_path = f"{INPUT_DIR}/ResilienceHub.gdb"

def load_gdb_layers():
    # List all layers in the geodatabase
    layers = fiona.listlayers(gdb_path)
    print(f"Available layers in geodatabase: {layers}")

    # Load required layers
    buildings = gpd.read_file(gdb_path, layer='NYC_Buildings')
    pois = gpd.read_file(gdb_path, layer='NYC_POIS')
    pofw = gpd.read_file(gdb_path, layer='NYC_POFW')
    # nsi = gpd.read_file(gdb_path, layer='NYC_NSI')

    return buildings, pois, pofw

# Load the data
# buildings, pois, pofw, nsi = load_gdb_layers()
buildings, pois, pofw = load_gdb_layers()

print("\nDataset Summary:")
print(f"Number of buildings: {len(buildings):,}")
print(f"Number of Places of Interest: {len(pois):,}")
print(f"Number of Places of Worship: {len(pofw):,}")

# Display sample of each dataset
print("\nBuildings sample columns:")
print(buildings.columns.tolist())
print("\nPOIs sample columns:")
print(pois.columns.tolist())


Available layers in geodatabase: ['NYC_POIS', 'NYC_NSI', 'NYC_Buildings', 'NYC_BBox', 'NYC_POFW', 'FDNY_Firehouses']

Dataset Summary:
Number of buildings: 1,084,413
Number of Places of Interest: 25,292
Number of Places of Worship: 1,106

Buildings sample columns:
['base_bbl', 'bin', 'cnstrct_yr', 'doitt_id', 'feat_code', 'geomsource', 'groundelev', 'heightroof', 'date_lstmo', 'time_lstmo', 'lststatype', 'mpluto_bbl', 'name', 'Shape_Length', 'Shape_Area', 'geometry']

POIs sample columns:
['osm_id', 'code', 'fclass', 'name', 'geometry']


In [14]:
# # Set paths and load data
# import geopandas as gpd
# import fiona
# import gc

# # Set base input directory
# INPUT_DIR = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/06 Scripts/ResilienceHub/Input'

# # Derive paths for input files
# gdb_path = f"{INPUT_DIR}/ResilienceHub.gdb"

# # Clear the Python kernel's memory cache
# gc.collect()

# def load_gdb_layers():
#     try:
#         # List all layers in the geodatabase
#         layers = fiona.listlayers(gdb_path)
#         print(f"Available layers in geodatabase: {layers}")

#         # Load required layers
#         buildings = gpd.read_file(gdb_path, layer='NYC_Buildings')
#         pois = gpd.read_file(gdb_path, layer='NYC_POIS')
#         pofw = gpd.read_file(gdb_path, layer='NYC_POFW')
#         firehouses = gpd.read_file(gdb_path, layer='FDNY_Firehouses', driver='OpenFileGDB')

#         # Verify the data load
#         print("\nVerifying data load:")
#         for name, data in [('buildings', buildings), ('pois', pois), 
#                           ('pofw', pofw), ('firehouses', firehouses)]:
#             print(f"\n{name.upper()} columns:")
#             print(data.columns.tolist())
#             print(f"Number of records: {len(data)}")

#         return buildings, pois, pofw, firehouses

#     except Exception as e:
#         print(f"Error loading data: {str(e)}")
#         raise

# # Load the data
# buildings, pois, pofw, firehouses = load_gdb_layers()

# # Print summary statistics
# print("\nDataset Summary:")
# print(f"Number of buildings: {len(buildings):,}")
# print(f"Number of Places of Interest: {len(pois):,}")
# print(f"Number of Places of Worship: {len(pofw):,}")
# print(f"Number of Firehouses: {len(firehouses):,}")


In [15]:
# # Diagnostic code to examine firehouses dataset
# print("\nFirehouses Dataset Information:")
# print("Columns:")
# print(firehouses.columns.tolist())
# print("\nShape:", firehouses.shape)
# print("\nData Types:")
# print(firehouses.dtypes)

# # Try to display all columns, not just geometry
# print("\nFirst few rows with all columns:")
# print(firehouses.head().to_string())


# Filter and prepare candidate buildings

In [16]:
# Filter and prepare candidate buildings
def filter_pois(pois):
    target_classes = [
        'arts_centre', 'college', 'community_centre', 
        'kindergarten', 'library', 'school', 'shelter'
    ]
    filtered_pois = pois[pois['fclass'].isin(target_classes)].copy()
    print(f"\nFiltered POIs by class:")
    print(filtered_pois['fclass'].value_counts())
    return filtered_pois

def merge_candidates(filtered_pois, pofw):
    # Prepare POFW by adding prefix to fclass
    pofw_prep = pofw.copy()
    pofw_prep['fclass'] = 'pofw_' + pofw_prep['fclass']

    # Select columns to keep
    cols_to_keep = ['fclass', 'name', 'geometry']

    # Merge datasets
    candidates = pd.concat([
        filtered_pois[cols_to_keep],
        pofw_prep[cols_to_keep]
    ], ignore_index=True)

    # Add ObjectID
    candidates['ObjectID'] = candidates.index + 1

    print(f"\nMerged candidates by class:")
    print(candidates['fclass'].value_counts())
    return candidates

# Process candidates
filtered_pois = filter_pois(pois)
candidate_points = merge_candidates(filtered_pois, pofw)

# Select building footprints
candidate_buildings = gpd.sjoin(
    buildings,
    candidate_points,
    how='inner',
    predicate='contains'
)

print(f"\nFinal candidate buildings:")
print(f"Total buildings selected: {len(candidate_buildings):,}")
print("\nDistribution by facility type:")
print(candidate_buildings['fclass'].value_counts())



Filtered POIs by class:
fclass
school              718
kindergarten        115
arts_centre          83
community_centre     68
library              56
college              22
shelter               7
Name: count, dtype: int64

Merged candidates by class:
fclass
pofw_christian                731
school                        718
kindergarten                  115
pofw_jewish                   112
arts_centre                    83
pofw_christian_catholic        71
community_centre               68
pofw_christian_lutheran        59
library                        56
pofw_christian_methodist       50
pofw_muslim                    35
college                        22
pofw_buddhist                  18
pofw_hindu                      9
pofw_muslim_sunni               9
shelter                         7
pofw_christian_orthodox         4
pofw_christian_protestant       3
pofw_christian_anglican         2
pofw_sikh                       2
pofw_christian_evangelical      1
Name: count, dtype: int6

In [17]:
# # Filter and prepare candidate buildings
# def filter_pois(pois):
#     target_classes = [
#         'arts_centre', 'college', 'community_centre', 
#         'kindergarten', 'library', 'school', 'shelter'
#     ]
#     filtered_pois = pois[pois['fclass'].isin(target_classes)].copy()
#     print(f"\nFiltered POIs by class:")
#     print(filtered_pois['fclass'].value_counts())
#     return filtered_pois

# def prepare_firehouses(firehouses):
#     # Create a copy of the firehouses dataset
#     firehouses_prep = firehouses.copy()

#     # Add required columns with appropriate values
#     firehouses_prep['fclass'] = 'Firehouse'
#     firehouses_prep['name'] = firehouses_prep['FacilityName']

#     # Select only needed columns
#     cols_to_keep = ['fclass', 'name', 'geometry']
#     firehouses_prep = firehouses_prep[cols_to_keep]

#     print(f"\nPrepared firehouses:")
#     print(f"Total firehouses: {len(firehouses_prep)}")
#     return firehouses_prep

# def merge_candidates(filtered_pois, pofw, firehouses):
#     # Prepare POFW by adding prefix to fclass
#     pofw_prep = pofw.copy()
#     pofw_prep['fclass'] = 'pofw_' + pofw_prep['fclass']

#     # Prepare firehouses
#     firehouses_prep = prepare_firehouses(firehouses)

#     # Select columns to keep
#     cols_to_keep = ['fclass', 'name', 'geometry']

#     # Merge all three datasets
#     candidates = pd.concat([
#         filtered_pois[cols_to_keep],
#         pofw_prep[cols_to_keep],
#         firehouses_prep[cols_to_keep]
#     ], ignore_index=True)

#     # Add ObjectID
#     candidates['ObjectID'] = candidates.index + 1

#     print(f"\nMerged candidates by class:")
#     print(candidates['fclass'].value_counts())
#     return candidates

# # Process candidates
# filtered_pois = filter_pois(pois)
# candidate_points = merge_candidates(filtered_pois, pofw, firehouses)

# # Select building footprints
# candidate_buildings = gpd.sjoin(
#     buildings,
#     candidate_points,
#     how='inner',
#     predicate='contains'
# )

# print(f"\nFinal candidate buildings:")
# print(f"Total buildings selected: {len(candidate_buildings):,}")
# print("\nDistribution by facility type:")
# print(candidate_buildings['fclass'].value_counts())


# Run the analysis


In [25]:
# Run the solar analysis
import time
import os
from solar_analysis import analyze_solar_potential
from run_solar_analysis import run_analysis
import multiprocessing as mp

print("\n=== Starting Solar Analysis with Import Verification ===")

# Create analyzer instance to verify class access
try:
    analyzer = solar_analysis.SolarAnalyzer()
    print("\nSolarAnalyzer initialization test:")
    print(f"Monthly radiation data: {analyzer._monthly_radiation}")
    print(f"Annual radiation calculation: {analyzer._annual_radiation:.2f} kWh/m²/year")
except Exception as e:
    print(f"Error creating SolarAnalyzer: {str(e)}")

# First, verify input data
print("\nInput Data Verification:")
print(f"Number of candidate buildings: {len(candidate_buildings)}")
print(f"Number of total buildings: {len(buildings)}")
print("\nSample of first building data:")
sample_building = candidate_buildings.iloc[0]
print(f"- Shape_Area: {sample_building['Shape_Area']}")
print(f"- Height: {sample_building['heightroof']}")

# Run analysis with both required arguments
analyzed_buildings = analyze_solar_potential(candidate_buildings, buildings)

if analyzed_buildings is None:
    print("\nERROR: Analysis failed to produce valid results")
    exit()

# Enhanced debug prints
print("\n=== Detailed Analysis Results ===")
print("\nBasic Statistics:")
print(f"Total buildings processed: {len(analyzed_buildings)}")
print(f"Columns present: {analyzed_buildings.columns.tolist()}")

# Detailed analysis of first building
print("\nDetailed Analysis of First Building:")
first_building = analyzed_buildings.iloc[0]
print("\nInput values:")
print(f"- Area: {first_building['Shape_Area']} sq ft")
print(f"- Height: {first_building['heightroof']} ft")
print("\nCalculated values:")
print(f"- Effective area: {first_building['effective_area']:.2f} m²")
print(f"- Shadow factor: {first_building['shadow_factor']:.2f}")
print(f"- Annual radiation: {first_building['annual_radiation']:.2f} kWh/m²/year")
print(f"- Solar potential: {first_building['solar_potential']:.2f} kWh/year")
print(f"- Peak power: {first_building['peak_power']:.2f} kW")

# Analysis of all buildings
print("\n=== Overall Analysis ===")
positive_potential = analyzed_buildings[analyzed_buildings['solar_potential'] > 0.01]
print(f"\nBuildings with positive solar potential: {len(positive_potential)}")

if len(positive_potential) > 0:
    print("\nSolar Potential Statistics:")
    print(f"Total potential: {positive_potential['solar_potential'].sum():,.0f} kWh/year")
    print(f"Average potential: {positive_potential['solar_potential'].mean():,.0f} kWh/year")
    print(f"Maximum potential: {positive_potential['solar_potential'].max():,.0f} kWh/year")
else:
    print("\n=== Diagnostic Information ===")
    print("\nKey Metrics:")
    print(f"Buildings with valid height: {(analyzed_buildings['heightroof'] > 0).sum()}")
    print(f"Buildings with valid area: {(analyzed_buildings['Shape_Area'] > 0).sum()}")
    print(f"Buildings with valid shadow factor: {(analyzed_buildings['shadow_factor'] > 0).sum()}")
    print(f"Buildings with valid annual radiation: {(analyzed_buildings['annual_radiation'] > 0).sum()}")

# Save debug information to file
debug_output = Path('results') / f'solar_analysis_debug_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt'
debug_output.parent.mkdir(exist_ok=True)

with open(debug_output, 'w') as f:
    f.write("=== Solar Analysis Debug Output ===\n")
    f.write(f"Total buildings: {len(analyzed_buildings)}\n")
    f.write(f"Buildings with positive potential: {len(positive_potential)}\n")
    f.write("\nColumn Statistics:\n")
    for col in analyzed_buildings.columns:
        if analyzed_buildings[col].dtype in [np.float64, np.int64]:
            f.write(f"\n{col}:\n")
            f.write(f"- Range: {analyzed_buildings[col].min()} to {analyzed_buildings[col].max()}\n")
            f.write(f"- Mean: {analyzed_buildings[col].mean()}\n")
            f.write(f"- Non-null count: {analyzed_buildings[col].count()}\n")

print(f"\nDebug information saved to: {debug_output}")


=== Starting Solar Analysis with Import Verification ===
Error creating SolarAnalyzer: name 'solar_analysis' is not defined

Input Data Verification:
Number of candidate buildings: 1776
Number of total buildings: 1084413

Sample of first building data:
- Shape_Area: 3377.7584572166847
- Height: 38.0

=== Starting Optimized Solar Analysis ===


NameError: name 'time' is not defined

# Web Map

In [ ]:
# Import custom module
from solar_mapping import create_detailed_solar_map

# Create and display the map
solar_map = create_detailed_solar_map(analyzed_buildings)

# Save the map
output_dir = Path('results')
output_dir.mkdir(exist_ok=True)
map_file = output_dir / 'solar_potential_map.html'
solar_map.save(str(map_file))

# Display the map in the notebook
solar_map

NameError: name 'analyzed_buildings' is not defined

# heat analysis

In [ ]:
# import numpy as np
# import rasterio
# from rasterio.mask import mask
# from rasterio.io import MemoryFile
# import pandas as pd
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings('ignore')

# def kelvin_to_celsius(k):
#     """Convert Kelvin to Celsius"""
#     return k - 273.15

# def kelvin_to_fahrenheit(k):
#     """Convert Kelvin to Fahrenheit"""
#     return (k - 273.15) * 9/5 + 32

# def create_empty_stats(idx):
#     """Create empty statistics dictionary"""
#     return {
#         'building_id': idx,
#         'pixel_count': 0,
#         'unique_values': 0,
#         'median_temperature_c': np.nan,
#         'mean_temperature_c': np.nan,
#         'max_temperature_c': np.nan,
#         'min_temperature_c': np.nan,
#         'temperature_range_c': np.nan,
#         'median_temperature_f': np.nan,
#         'mean_temperature_f': np.nan,
#         'max_temperature_f': np.nan,
#         'min_temperature_f': np.nan,
#         'temperature_range_f': np.nan,
#         'buffer_area_m2': 0
#     }

# def process_chunk(gdf_chunk, raster_src):
#     """Process a chunk of buildings using a raster source"""
#     results = []
#     debug_count = 0

#     # Get the raster's transform and CRS
#     transform = raster_src.transform

#     for idx, row in gdf_chunk.iterrows():
#         try:
#             # Create a smaller buffer
#             buffer_geom = row.geometry.buffer(30)  # 30 meters buffer

#             # Convert to GeoJSON-like dict
#             geom = [buffer_geom.__geo_interface__]

#             try:
#                 # Get window for the geometry
#                 building_data, out_transform = mask(raster_src, 
#                                                  geom, 
#                                                  crop=True, 
#                                                  all_touched=False,
#                                                  nodata=np.nan)

#                 # Flatten and remove NaN values
#                 valid_data = building_data.flatten()
#                 valid_data = valid_data[~np.isnan(valid_data)]

#                 if len(valid_data) > 0:
#                     # Debug printing for first few buildings
#                     if debug_count < 5:
#                         print(f"\nBuilding {idx} data summary:")
#                         print(f"Geometry bounds: {buffer_geom.bounds}")
#                         print(f"Shape of masked data: {building_data.shape}")
#                         print(f"Number of valid pixels: {len(valid_data)}")
#                         print(f"Raw Kelvin values - Min: {valid_data.min():.2f}, Max: {valid_data.max():.2f}")
#                         print(f"Converted to Celsius - Min: {kelvin_to_celsius(valid_data.min()):.2f}, "
#                               f"Max: {kelvin_to_celsius(valid_data.max()):.2f}")
#                         print(f"Unique values: {len(np.unique(valid_data))}")
#                         debug_count += 1

#                     # Convert and store statistics
#                     valid_data_c = kelvin_to_celsius(valid_data)

#                     stats = {
#                         'building_id': idx,
#                         'pixel_count': len(valid_data),
#                         'unique_values': len(np.unique(valid_data)),
#                         'median_temperature_c': float(np.median(valid_data_c)),
#                         'mean_temperature_c': float(np.mean(valid_data_c)),
#                         'max_temperature_c': float(np.max(valid_data_c)),
#                         'min_temperature_c': float(np.min(valid_data_c))
#                     }

#                     # Calculate ranges
#                     stats['temperature_range_c'] = stats['max_temperature_c'] - stats['min_temperature_c']

#                     # Add Fahrenheit conversions
#                     stats.update({
#                         'median_temperature_f': (stats['median_temperature_c'] * 9/5) + 32,
#                         'mean_temperature_f': (stats['mean_temperature_c'] * 9/5) + 32,
#                         'max_temperature_f': (stats['max_temperature_c'] * 9/5) + 32,
#                         'min_temperature_f': (stats['min_temperature_c'] * 9/5) + 32,
#                         'temperature_range_f': stats['temperature_range_c'] * 9/5
#                     })

#                 else:
#                     raise ValueError("No valid data found in masked region")

#             except Exception as e:
#                 print(f"Masking error for building {idx}: {str(e)}")
#                 stats = create_empty_stats(idx)

#             # Add buffer area regardless of success
#             stats['buffer_area_m2'] = float(buffer_geom.area)
#             results.append(stats)

#         except Exception as e:
#             print(f"Error processing building {idx}: {str(e)}")
#             results.append(create_empty_stats(idx))

#     return results

# def create_detailed_summary(gdf):
#     """Create detailed summary statistics"""
#     total_sites = len(gdf)
#     valid_sites = gdf['mean_temperature_c'].notna().sum()

#     summary = {
#         'total_sites': total_sites,
#         'valid_sites': valid_sites,
#         'coverage_percentage': (valid_sites/total_sites)*100 if total_sites > 0 else 0
#     }

#     if valid_sites > 0:
#         # Data coverage statistics
#         summary['coverage_stats'] = {
#             'total_pixels': int(gdf['pixel_count'].sum()),
#             'mean_pixels_per_building': float(gdf['pixel_count'].mean()),
#             'median_pixels_per_building': float(gdf['pixel_count'].median()),
#             'mean_unique_values': float(gdf['unique_values'].mean())
#         }

#         # Temperature statistics
#         temp_stats = {}
#         for unit, cols in [('celsius', '_c'), ('fahrenheit', '_f')]:
#             temp_stats[unit] = {
#                 'mean': float(gdf[f'mean_temperature{cols}'].mean()),
#                 'median': float(gdf[f'median_temperature{cols}'].median()),
#                 'max': float(gdf[f'max_temperature{cols}'].max()),
#                 'min': float(gdf[f'min_temperature{cols}'].min()),
#                 'std': float(gdf[f'mean_temperature{cols}'].std()),
#                 'avg_range': float(gdf[f'temperature_range{cols}'].mean())
#             }
#         summary['temperature_stats'] = temp_stats

#         # Area statistics
#         summary['area_stats'] = {
#             'total_buffer_area': float(gdf['buffer_area_m2'].sum()),
#             'mean_buffer_area': float(gdf['buffer_area_m2'].mean()),
#             'median_buffer_area': float(gdf['buffer_area_m2'].median())
#         }

#         # Temperature distribution
#         for unit, col in [('celsius', 'mean_temperature_c'), 
#                          ('fahrenheit', 'mean_temperature_f')]:
#             # Calculate quintiles
#             quintiles = gdf[col].quantile([0, 0.2, 0.4, 0.6, 0.8, 1.0])

#             try:
#                 temp_bins = pd.cut(gdf[col], 
#                                  bins=quintiles,
#                                  labels=[f'{quintiles[i]:.1f} to {quintiles[i+1]:.1f}' 
#                                        for i in range(len(quintiles)-1)],
#                                  include_lowest=True)

#                 temp_dist = temp_bins.value_counts().sort_index()

#                 summary[f'temperature_distribution_{unit}'] = {
#                     str(bin_label): {
#                         'count': int(count),
#                         'percentage': float((count/valid_sites)*100)
#                     } for bin_label, count in temp_dist.items()
#                 }

#                 # Add quintile values
#                 summary[f'temperature_quintiles_{unit}'] = {
#                     f'Q{i}': float(val) for i, val in enumerate(quintiles)
#                 }

#             except Exception as e:
#                 print(f"\nWarning: Distribution calculation failed for {unit}")
#                 print(f"Data summary for {col}:")
#                 print(gdf[col].describe())
#                 print(f"Error: {str(e)}")

#     return summary

# def print_summary(summary):
#     """Print formatted summary statistics"""
#     print("\n=== Analysis Summary ===")
#     print(f"Total buildings analyzed: {summary['total_sites']:,}")
#     print(f"Valid readings: {summary['valid_sites']:,} ({summary['coverage_percentage']:.1f}%)")

#     if summary['valid_sites'] > 0:
#         print("\nCoverage Statistics:")
#         cov_stats = summary['coverage_stats']
#         print(f"Total pixels analyzed: {cov_stats['total_pixels']:,}")
#         print(f"Mean pixels per building: {cov_stats['mean_pixels_per_building']:.1f}")
#         print(f"Mean unique values per building: {cov_stats['mean_unique_values']:.1f}")

#         print("\nTemperature Statistics:")

#         for unit in ['celsius', 'fahrenheit']:
#             symbol = '°C' if unit == 'celsius' else '°F'
#             print(f"\n{unit.capitalize()}:")
#             temp_stats = summary['temperature_stats'][unit]
#             print(f"Mean temperature: {temp_stats['mean']:.1f}{symbol}")
#             print(f"Median temperature: {temp_stats['median']:.1f}{symbol}")
#             print(f"Maximum temperature: {temp_stats['max']:.1f}{symbol}")
#             print(f"Minimum temperature: {temp_stats['min']:.1f}{symbol}")
#             print(f"Standard deviation: {temp_stats['std']:.1f}{symbol}")
#             print(f"Average temperature range: {temp_stats['avg_range']:.1f}{symbol}")

#         print("\nArea Statistics:")
#         area_stats = summary['area_stats']
#         print(f"Total buffer area: {area_stats['total_buffer_area']:,.0f} sq meters")
#         print(f"Mean buffer area: {area_stats['mean_buffer_area']:,.0f} sq meters")

#         print("\nTemperature Distribution:")
#         for unit in ['celsius', 'fahrenheit']:
#             symbol = '°C' if unit == 'celsius' else '°F'
#             print(f"\n{unit.capitalize()} Quintiles:")
#             quintiles = summary[f'temperature_quintiles_{unit}']
#             for q, val in quintiles.items():
#                 print(f"{q}: {val:.1f}{symbol}")

#             print(f"\n{unit.capitalize()} Distribution:")
#             for bin_range, data in summary[f'temperature_distribution_{unit}'].items():
#                 print(f"{bin_range}{symbol}: {data['count']:,} buildings ({data['percentage']:.1f}%)")
#     else:
#         print("\nNo valid temperature readings found")

# def analyze_heat_exposure(gdf, chunk_size=50):
#     """Process buildings in chunks using rasterio source"""
#     temp_raster_path = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/02 Data/00 Sources & Packages/Heat/Landsat9_ThermalComposite_ST_B10_2020-2023.tif'

#     print("Loading raster data...")
#     with rasterio.open(temp_raster_path) as src:
#         # Convert GDF to raster CRS first
#         gdf = gdf.to_crs(src.crs)

#         # Process in chunks
#         results = []
#         num_chunks = len(gdf) // chunk_size + (1 if len(gdf) % chunk_size else 0)

#         print(f"Processing {len(gdf)} buildings in {num_chunks} chunks...")
#         for i in tqdm(range(0, len(gdf), chunk_size)):
#             chunk = gdf.iloc[i:i+chunk_size]
#             chunk_results = process_chunk(chunk, src)
#             results.extend(chunk_results)

#     # Convert results to DataFrame
#     results_df = pd.DataFrame(results).set_index('building_id')

#     # Merge results with original GDF
#     for column in results_df.columns:
#         gdf[column] = results_df[column]

#     # Create and print detailed summary
#     summary = create_detailed_summary(gdf)
#     print_summary(summary)

#     return gdf, summary

# # Usage example:
# candidate_buildings, summary = analyze_heat_exposure(candidate_buildings, chunk_size=50)


In [ ]:
# # Placeholder functions for additional analyses
# def analyze_social_vulnerability(candidate_buildings):
#     # Placeholder for social vulnerability analysis
#     pass

# def analyze_heat_exposure(candidate_buildings):
#     # Placeholder for extreme heat exposure analysis
#     pass

# def analyze_flood_risk(candidate_buildings):
#     # Placeholder for flood risk analysis
#     pass


In [ ]:
# # Combine all analyses and save final results
# def save_final_results(candidate_buildings, solar_results):
#     # Combine all analysis results
#     final_candidates = candidate_buildings.copy()

#     # Add solar analysis results
#     # ... [Add other analysis results when implemented] ...

#     # Generate filename with current date
#     current_date = datetime.now().strftime('%y%m%d')
#     output_name = f"RH_Candidates_{current_date}"

#     # Save to geodatabase (original approach - commented out)
#     # final_candidates.to_file(gdb_path, layer=output_name, driver="FileGDB")

#     # Save to shapefile instead
#     output_shapefile = f"RH_Candidates_{current_date}.shp"
#     final_candidates.to_file(output_shapefile)

#     return final_candidates

# final_results = save_final_results(candidate_buildings, solar_results)
